<a href="https://colab.research.google.com/github/Darjeel1ng/twitter-crawler/blob/main/twitter_crawler_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0 Setup

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd gdrive/MyDrive/COMP-440/COMP 440 S23: DEN/Emoji_Prediction
!ls

/content/gdrive/MyDrive/COMP-440/COMP 440 S23: DEN/Emoji_Prediction
bilstm_model  code  data


In [ ]:
!pip install snscrape
!pip install emoji
!pip install pycld3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 KB 2.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=7d0c2b844a9678025046e241b8727314fd159a09f0f27da1d013c5b0cefe428c
  Stored in directory: /root/.cache/pip/wheels/9a/b8/0f/f580817231cbf59f6ade9fd132ff60ada1de9f7dc85521f857
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 56.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import time
import datetime
from google.colab import files

import emoji
import cld3
import snscrape.modules.twitter as sntwitter

In [ ]:
pd.set_option('display.max_colwidth', 255)

# 1 Crawler

In [ ]:
def text_has_emoji(text):
  for character in text:
    if character in emoji.EMOJI_DATA:
      return True
  return False

In [ ]:
def get_emoji_data(query, limit, print_interval, save_interval):

  tweets = []
  prev_len = 0

  for tweet in sntwitter.TwitterSearchScraper(query).get_items():

    # print(vars(tweet))
    # break

    # Monitor
    if len(tweets) == 0:
      start_time = time.time()
    if prev_len != len(tweets): 
      prev_len = len(tweets)
      if len(tweets) % print_interval == 0:
        end_time = time.time()
        delta_time = end_time - start_time
        print_time = str(datetime.timedelta(seconds=round(delta_time, 0)))
        print(f'{len(tweets)}/{limit} - {print_time}')

      # Write csv
      if len(tweets) % save_interval == 0:
        emoji_data = pd.DataFrame(tweets, columns=['Text'])
        file_name = re.search(r':(.*):', emoji.EMOJI_DATA[query]['en']).group(1)
        emoji_data.to_csv(f'data/emoji/{file_name}.csv', index=False)
        print(f'Successfully saved in {file_name}.csv')
    
    # Crawler
    if len(tweets) == limit:
      print(f'{query} done!\n')
      break
    else:
      text = tweet.rawContent
      if text_has_emoji(text): # Get text that has emoji
        if cld3.get_language(text).language == 'en': # Get text that is in English
          tweets.append(text)


In [ ]:
run_emoji_list = [
    '😭', '😂', '❤️', '✨', '🙏', '😍', '🥰', '☺️', '👍', '😅', '😡', '😨', 
    '🥳', '🥵', '😤', '🤔', '🥲', '🤡', '🫡', '🔥', '🐰', '✅', '🫠', '🐣', 
    '🎉', '🥚', '⭐', '🤍', '👉', '😉', '🐇']

In [ ]:
for e in run_emoji_list[24:31]:
  get_emoji_data(e, 20000, 1000, 2000)

1000/20000 - 0:07:49


2000/20000 - 0:14:53
Successfully saved in party_popper.csv


3000/20000 - 0:24:59


4000/20000 - 0:33:32
Successfully saved in party_popper.csv


5000/20000 - 0:41:50
6000/20000 - 0:48:14
Successfully saved in party_popper.csv
7000/20000 - 0:52:58


8000/20000 - 0:57:01
Successfully saved in party_popper.csv
9000/20000 - 1:00:58


In [ ]:
emoji_finished = {
    '😭': '',
    '😂': '',
    '🥹': '1:40:22',
    '🤡': '1:15:13', 
    '💀': '0:48:37',
    '✔️': '2:09:08', 
    '🤣': '1:28:05', 
    '😋': '3:09:04',
    '👀': '1:02:36',
    '💩': '1:24:12'
}

# Resources

1. Detect if a string is English: [The WiLI benchmark dataset for written
language identification](https://arxiv.org/pdf/1801.07779.pdf)
2. Twitter crawler: [youtube](https://www.youtube.com/watch?v=jtIMnmbnOFo)
3. Emoji frequency: [emojipedia stats](https://emojipedia.org/stats/)
4. Emoji API: [documentation](https://carpedm20.github.io/emoji/docs/api.html#emoji.EMOJI_DATA)